# ctypes: C library call python function 


* Change the current working directory into `./demo`

In [1]:
%cd demo

F:\SEU\SEE\PySEE\home\notebook\demo


In [ ]:
%pwd

## 1 Bisection Methon Shared Library in C

* using `__cdecl`calling convention



In [2]:
%%file ./include/ourmath.h
#include <math.h>
// function definition
typedef double (*Fun)(double x);

double sign(double, double);
void bisect(Fun f, double, double, double, double *, int *);

Overwriting ./include/ourmath.h


In [3]:
%%file ./src/ourmath.c
/*
  Numerical Recipes http://numerical.recipes
*/ 
#include <math.h>
#include "ourmath.h"

double sign(double a, double b)
{
	if (b < 0.0)
		return (-fabs(a));
	else
		return (fabs(a));
}

void bisect(Fun f, double a, double b, double eps, double *root, int *ier)
{
	/*
	The program uses the bisection method to solve 	the equation
		f(x) = 0.
	The solution is to be in [a,b] and it is assumed 	that
		f(a)*f(b) <= 0.
	The solution is returned in root, and it is to	be in error by at most eps.
	
	ier is an error indicator.
	  If ier=0 on completion of the routine, then the 	solution has been computed satisfactorily.
	  If ier=1, then f(a)*f(b) was greater than 0, contrary to assumption.
*/

	const double zero = 0.0, one = 1.0, two = 2.0;
	double c, fa, fb, fc, sfa, sfb, sfc;

	// Initialize
	fa = (*f)(a);
	fb = (*f)(b);
	sfa = sign(one, fa);
	sfb = sign(one, fb);
	if (sfa * sfb > 0.0)
	{
		// The choice of a and b is in error
		*ier = 1;
		return;
	}

	// Create a new value of c, the midpoint of [a,b]
	while (1)
	{
		c = (a + b) / two;
		if (fabs(b - c) <= eps)
		{
			// c is an acceptable solution of f(x)=0
			*root = c;
			*ier = 0;
			return;
		}
		/* The value of c was not sufficiently accurate. 
			Begin a new iteration  */
		fc = (*f)(c);
		if (fc == zero)
		{
			// c is an acceptable solution of f(x)=0
			*root = c;
			*ier = 0;
			return;
		}
		sfc = sign(one, fc);
		if (sfb * sfc > zero)
		{
			//  The solution is in [a,c]
			b = c;
			sfb = sfc;
		}
		else
		{
			//  The solution is in [c,b]
			a = c;
			sfa = sfc;
		}
	}
}

Overwriting ./src/ourmath.c


In [4]:
!gcc -c -O3 -Wall -fPIC -o ./obj/ourmath.o  ./src/ourmath.c -I./include
!gcc -shared -o ./bin/libourmath.dll  ./obj/ourmath.o

## 2 Caller in C

In [5]:
%%file ./src/bisectApp.c

#include <stdio.h>
#include <string.h>
#include <stdlib.h>
#include "ourmath.h"

double fcn(double x)
{
	double result;
	result = x * x - 4;
	return (result);
}

int main()
{
	double xl, xr, epsilon, root;
	int ier;
	xl = 0.1;
	xr = 3.2;
	epsilon = 0.001;
	// Calculate root
	bisect(fcn, xl, xr, epsilon, &root, &ier);
	// Print answers
	printf("root = %14.7e  ier = %1d", root, ier);
	return 0;
}

Overwriting ./src/bisectApp.c


In [6]:
!gcc -o  ./bin/bisectApp ./src/bisectApp.c -L./bin/ -lourmath -I./include

In [7]:
!.\bin\bisectApp 

root = 2.0004150e+000  ier = 0


## 3 Python API 

There are several ways to load shared libraries into the Python process. 

**One way is to instantiate one of the following classes:**

**CDLL**

* Instances of this class represent loaded shared libraries. Functions in these libraries use the standard `C calling convention`, and are assumed to return int.
  
**WinDLL**

* Windows only: Instances of this class represent loaded shared libraries, functions in these libraries use the `stdcall calling convention`, and are assumed to return int by default

In [8]:
from ctypes import * 

api = CDLL('.\\bin\\libourmath.dll')
Fun = CFUNCTYPE(c_double, c_double)

bisect = api.bisect
bisect.argtypes = [Fun, c_double,c_double,c_double,POINTER(c_double),POINTER(c_int)]
bisect.restype = c_void_p

def findroot(fn,xl,xr,eps): 
    root=c_double()
    ier=c_int()
    bisect(fn,xl,xr,eps,byref(root),byref(ier))
    
    return root,ier

In [11]:
def fn(x):        
    return x**2-4

_fn = Fun(fn)
xl = c_double(0.2) 
xr = c_double(3)
eps=0.001

rt,ier=findroot(_fn,xl,xr,eps)
print(rt.value,ier.value)

1.99990234375 0


##  Reference

* [Numerical Recipes](http://numerical.recipes)

* [GSL - GNU Scientific Library](https://www.gnu.org/software/gsl/)

* [Intel Math Kernel Library](https://software.intel.com/en-us/mkl)